In [1]:
import torch
from google.colab import drive
drive.mount('/gdrive')
import sys
sys.path.append('/gdrive/My Drive/dl-pytorch/')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
class SIGNSDataset(Dataset):
  def __init__(self, base_dir, split="train", transform=None):
    path = os.path.join(base_dir, "{}_signs".format(split))
    files = os.listdir(path)

    self.filenames = [os.path.join(path, f) for f in files if f.endswith(".jpg")]

    self.targets = [int(f[0]) for f in files]

    self.transform = transform
  
  def __len__(self):
    return len(self.filenames)


  def __getitem__(self, idx):
    image = Image.open(self.filenames[idx])
    if self.transform:
      image = self.transform(image)

    return image, self.targets[idx]
    

  



In [3]:
#entrenamiento

import torch.optim as optim
device = torch.device('cuda')


In [4]:
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(), #data augmentation
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
     ]
)

In [6]:


trainset = SIGNSDataset('/gdrive/My Drive/dl-pytorch/datasets/64x64_SIGNS', split="train", transform = transform)
trainloader = DataLoader(trainset, batch_size = 32) 

In [7]:
valset = SIGNSDataset('/gdrive/My Drive/dl-pytorch/datasets/64x64_SIGNS', split="val", transform = transform)
valloader = DataLoader(valset, batch_size = 32)

In [8]:
testset = SIGNSDataset('/gdrive/My Drive/dl-pytorch/datasets/64x64_SIGNS', split="test", transform = transform)
testloader = DataLoader(testset, batch_size = 32)

In [9]:
dataloaders = {'train': trainloader,
               'val': valloader,
               'test': testloader}

In [10]:
dataloader = DataLoader(trainset, batch_size=32)  #data loader entrega por batches el dataset a la red neuronal

In [11]:
import torch.nn as nn
import torch.nn.functional as F #hay capas que no tienen parametros y se invocan como funciones
class Net(nn.Module):
  def __init__(self, num_channels):
    super(Net,self).__init__() #inizializacion
    # 3 capas convolucionales y 2 capas fully connected
    self.num_channels = num_channels
    
    self.conv1 = nn.Conv2d(3, self.num_channels, 3, stride = 1, padding = 1) #canales de entradqa, canales de salida, kernel size
    self.bn1 = nn.BatchNorm2d(self.num_channels)
    self.conv2 = nn.Conv2d(self.num_channels, self.num_channels*2, 3, stride = 1, padding = 1)
    self.bn2 = nn.BatchNorm2d(self.num_channels*2)
    self.conv3 = nn.Conv2d(self.num_channels*2, self.num_channels*4, 3, stride = 1, padding = 1)
    self.bn3 = nn.BatchNorm2d(self.num_channels*4)

    #en las capas sin parametros, maxpool: divide el tamaño de la imagen entre 2
    self.fc1 = nn.Linear(self.num_channels*4*8*8, self.num_channels*4) # entrada y salida            linear o fullyConnected
    self.fcbn1 = nn.BatchNorm1d(self.num_channels*4)
    self.fc2 = nn.Linear(self.num_channels*4, 6) # 6 es la salida final de la red
    
  def forward(self,x):
      #Empieza 3x64x64
      x = self.bn1(self.conv1(x)) # num_channels x 64 x 64
      x = F.relu(F.max_pool2d(x, 2)) # num_channels x 32 x 32
      x = self.bn2(self.conv2(x)) # num_channels*2 x 32 x32
      x = F.relu(F.max_pool2d(x, 2)) #num_channels*2 x 16 x 16
      x = self.bn3(self.conv3(x)) # num_channels*4 x16x16
      x = F.relu(F.max_pool2d(x, 2)) # num_channels*4 x 8 x 8

      #flatten
      x = x.view(-1, self.num_channels*4*8*8) #-1 flatten, size

      #fc
      x = F.relu(self.fcbn1( self.fc1(x)))
      x = F.dropout(x, p  = 0.8, training=True)
      x = self.fc2(x)

      #log_softmax

      x = F.log_softmax(x, dim=1)

      return x             

In [12]:
net = Net(32).to(device) #el parametro es el numero de canales

loss_fn = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum = 0.9)

In [13]:
#suma de valores y cantidad o tamaño de valores
class RunningMetric():
  def __init__(self):
      self.S = 0
      self.N = 0
  def update(self, val, size):
    self.S+=val
    self.N+= size

  def __call__(self):
    return self.S/float(self.N)

In [14]:
num_epoch = 100 #pasadas dataset
#loop de entrenamiento

def train_and_evaluate(model, optimizer, loss_fn, dataloaders,device, num_epoch = 10, lr = 0.001):
  for g in optimizer.param_groups:
    g['lr'] = lr
  for epoch in range(num_epoch):
    print('Epoch {} / {} '.format(epoch+1, num_epoch))
    print('-'*10)

    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()
      running_loss = RunningMetric() #perdida
      running_acc = RunningMetric() #precision

      for inputs, targets in dataloaders[phase]:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad() #reset
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs)
          _, preds = torch.max(outputs,1)
          loss = loss_fn(outputs, targets)
          if phase == 'train':
            loss.backward() # gradientes calculados automaticamente
            optimizer.step() #Actualiza parametros

        batch_size = inputs.size()[0]
        running_loss.update(loss.item()*batch_size, batch_size)

        running_acc.update(torch.sum(preds == targets).float(), batch_size)
      #print("Loss: {:.4f} Acc: {:.4f} ".format(running_loss(), running_acc()))
      print("Loss{}: {:.4f} Acc{}: {:.4f} lr : {}".format(phase,running_loss(),phase,running_acc(),lr))



  return model

In [15]:
import random
lrs = [10**(-random.randint(3,7)) for  _ in range(3)]

In [16]:
for lr in lrs:
  train_and_evaluate(net, optimizer, loss_fn, dataloaders, device, 10, lr)


Epoch 1 / 10 
----------
Losstrain: 2.1049 Acctrain: 0.1655 lr : 1e-06
Lossval: 1.9028 Accval: 0.1944 lr : 1e-06
Epoch 2 / 10 
----------
Losstrain: 2.1009 Acctrain: 0.1759 lr : 1e-06
Lossval: 2.0623 Accval: 0.1620 lr : 1e-06
Epoch 3 / 10 
----------
Losstrain: 2.0894 Acctrain: 0.1817 lr : 1e-06
Lossval: 2.0251 Accval: 0.1667 lr : 1e-06
Epoch 4 / 10 
----------
Losstrain: 2.0662 Acctrain: 0.1840 lr : 1e-06
Lossval: 2.0812 Accval: 0.1620 lr : 1e-06
Epoch 5 / 10 
----------
Losstrain: 2.0312 Acctrain: 0.1898 lr : 1e-06
Lossval: 2.1039 Accval: 0.1528 lr : 1e-06
Epoch 6 / 10 
----------
Losstrain: 2.0268 Acctrain: 0.1794 lr : 1e-06
Lossval: 2.0704 Accval: 0.1713 lr : 1e-06
Epoch 7 / 10 
----------
Losstrain: 2.0786 Acctrain: 0.1713 lr : 1e-06
Lossval: 2.0041 Accval: 0.1806 lr : 1e-06
Epoch 8 / 10 
----------
Losstrain: 2.0453 Acctrain: 0.1852 lr : 1e-06
Lossval: 2.0835 Accval: 0.1898 lr : 1e-06
Epoch 9 / 10 
----------
Losstrain: 2.0455 Acctrain: 0.1852 lr : 1e-06
Lossval: 2.0741 Accval: 0